In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

## Day 47 Lecture 2 Assignment

In this assignment, we will perform K-Medoids clustering using responses to a survey about student life at a university.

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.spatial.distance import pdist, squareform
from pyclustering.cluster.kmedoids import kmedoids
import random

<IPython.core.display.Javascript object>

This dataset consists of 35 binary features, each corresponding to a yes/no question that characterizes the student taking the survey.

This dataset contains a large number of features, each corresponding to a survey question. The feature name summarizes the survey question, so we will not list them all out here.

Load the dataset.

In [3]:
# answer goes here
df = pd.read_csv('data/student_life_survey.csv')

df.head()


,Q2-Participated in Societies and Interest Groups,Q2-Participated in Clubs,"Q2-Participated in Halls, JCRCs and/or Residential College CSCs",Q2-Participated in University organised events,Q3-Interested in Arts & Culture,Q3-Interested in Science & Technology,Q3-Interested in Research and independent study,Q3-Interested in Sports,"Q3-Interested in Other competitions (eg case, debates)",Q3-Interested in Entrepreneurship,...,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,response_id
0,0,1,0,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1
1,0,1,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,2
2,0,0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,3
3,1,1,1,1,0,1,1,0,0,0,...,1,0,1,1,1,1,0,1,0,4
4,1,0,1,1,0,1,1,0,0,1,...,1,1,0,1,0,0,0,1,0,5


<IPython.core.display.Javascript object>

For our analysis, we will focus on a specific subset of the survey that is focused on stress. These questions all begin with the string 'Q5'. Filter the columns that meet this criteria (should be 10 in total).

In [4]:
q5 = df.filter(like="Q5")
q5

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0
3,1,1,0,1,1,1,1,0,1,0
4,0,1,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
2953,0,1,1,0,0,1,0,0,0,0
2954,0,1,0,0,0,0,0,0,0,0
2955,0,1,0,0,0,0,0,0,0,0
2956,0,1,0,0,0,0,1,1,0,0


<IPython.core.display.Javascript object>

The pyclustering implementation of kmedoids supports a variety of distance metrics, but they are primarily for numeric data. We will be using SMC/Hamming dissimilarity and precomputing the similarity matrix (an alternative would be to specify a user-defined function, which you are welcome to try in addition).

We'll assume for the next step that a pair of negative values (i.e. both responses are "no") is as informative as a pair of positive values. Compute the full distance/dissimilarity matrix for the survey data using matching/hamming distance.

In [5]:
# answer goes here
dist_mat = squareform(pdist(q5, metric="hamming"))
dist_mat 


array([[0. , 0. , 0.1, ..., 0. , 0.2, 0.2],
       [0. , 0. , 0.1, ..., 0. , 0.2, 0.2],
       [0.1, 0.1, 0. , ..., 0.1, 0.1, 0.1],
       ...,
       [0. , 0. , 0.1, ..., 0. , 0.2, 0.2],
       [0.2, 0.2, 0.1, ..., 0.2, 0. , 0.2],
       [0.2, 0.2, 0.1, ..., 0.2, 0.2, 0. ]])

<IPython.core.display.Javascript object>

Using the dissimilarity matrix, perform kmedoids clustering using k=2. Set the initial medoids randomly. Note that pyclustering expects the distance matrix to be a numpy array; a pandas dataframe may cause errors. 

Which survey responses are chosen as the cluster representatives? Print out the details of these responses.

In [6]:
dist_mat_df = pd.DataFrame(dist_mat)

<IPython.core.display.Javascript object>

In [7]:
tall_dist_df = pd.melt(dist_mat_df.reset_index(), id_vars="index")
tall_dist_df.columns = ["student_1", "student_2", "hamming_dist"]

# removing redundant info
tall_dist_df = tall_dist_df[tall_dist_df["student_1"] < tall_dist_df["student_2"]]
print(tall_dist_df.shape)
tall_dist_df.head(3)

(4373403, 3)


,student_1,student_2,hamming_dist
2958,0,1,0.0
5916,0,2,0.1
5917,1,2,0.1


<IPython.core.display.Javascript object>

In [8]:
tall_dist_df

,student_1,student_2,hamming_dist
2958,0,1,0.0
5916,0,2,0.1
5917,1,2,0.1
8874,0,3,0.6
8875,1,3,0.6
...,...,...,...
8749758,2952,2957,0.5
8749759,2953,2957,0.2
8749760,2954,2957,0.2
8749761,2955,2957,0.2


<IPython.core.display.Javascript object>

In [9]:
pick_med = tall_dist_df[tall_dist_df["hamming_dist"] > 0.8].sample()
med_1 = pick_med["student_1"].values.astype(int)
med_2 = pick_med["student_2"].values.astype(int)
init_medoids = [med_1[0], med_2[0]]

<IPython.core.display.Javascript object>

In [10]:
init_medoids

[705, 2179]

<IPython.core.display.Javascript object>

In [11]:
# answer goes here
# k = 2
#np.random.seed(34)

# nrows = dist_mat.shape[0]
# init_medoids = np.random.randint(0, nrows+1, k)
# init_medoids = tall_dist_df[tall_dist_df["hamming_dist"] > 0.8].sample().iloc[:, :2].values

# init_medoids
kmed = kmedoids(
    dist_mat, initial_index_medoids=init_medoids, data_type="distance_matrix"
)

kmed.process()


<IPython.core.display.Javascript object>

In [12]:
init_medoids

[705, 2179]

<IPython.core.display.Javascript object>

In [13]:
medoid_idxs = kmed.get_medoids()
medoid_idxs

[0, 65]

<IPython.core.display.Javascript object>

In [14]:
labels = kmed.predict(dist_mat)
q5["label"] = labels
q5.iloc[medoid_idxs, :]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,label
0,0,1,0,0,0,0,0,0,0,0,0
65,1,1,0,0,1,0,0,1,1,0,1


<IPython.core.display.Javascript object>

If you run the previous cell a few times, you'll probably notice that the medoids are very sensitive to initialization. A common approach to produce well-separated clusters is to choose initial centroids that are far apart. Re-run the previous process, except with a random pair of centroids that have a dissimilarity of 0.8 or higher. Are the results more stable now? How would you describe the typical clusters you see?

In [15]:
# answer goes here

clst_avg = q5.groupby("label").mean().T
clst_avg.style.background_gradient(axis=1)



label,0,1
Q5-Stressed about Adjustment issues,0.142322,0.588613
Q5-Stressed about Academic issues,0.918138,0.929293
Q5-Stressed about Financial issues,0.301231,0.531680
Q5-Stressed about Family issues,0.061530,0.294766
Q5-Stressed about Friendships,0.083467,0.589532
Q5-Stressed about Romantic relationships,0.075441,0.320478
Q5-Stressed about Health related issues,0.070626,0.289256
Q5-Stressed about Career related issues,0.368646,0.603306
"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",0.106474,0.561983
Q5-Stressed about Others,0.013376,0.014692


<IPython.core.display.Javascript object>